# 📘 Архитектуры Агентов 20: Debate & Red-teaming (Дебаты и Adversarial проверка)

В этом блокноте мы реализуем архитектуру **Debate + Red-teaming**, где несколько агентов занимают противоположные позиции для всестороннего анализа проблемы. В отличие от простых систем "вопрос-ответ", здесь мы намеренно вводим конфликт точек зрения, чтобы выявить слабые места, галлюцинации или логические ошибки.

Ключевая особенность нашей реализации — **обязательное цитирование**. Агенты не просто спорят, они обязаны предоставлять ссылки на источники (симулированные или реальные), а судья проверяет валидность этих ссылок.

### Определение
**Debate / Red-teaming** архитектура состоит из трех основных ролей:
1. **Advocate (Защитник):** Предлагает аргументы в поддержку тезиса или решения.
2. **Adversary (Оппонент/Red-teamer):** Активно ищет недостатки, ошибки в рассуждениях или ложную информацию.
3. **Judge (Судья):** Беспристрастный арбитр, который оценивает качество аргументов, проверяет источники и выносит вердикт.

### Высокоуровневый рабочий процесс

1.  **Аргументация:** Advocate выдвигает тезис с доказательствами.
2.  **Критика:** Adversary атакует этот тезис, указывая на слабые места.
3.  **Защита:** Advocate отвечает на критику, уточняя позицию.
4.  **Вердикт:** После N раундов Judge анализирует историю спора и выносит решение.

### Применения
*   **Fact-checking:** Проверка утверждений на достоверность.
*   **Risk Analysis:** Поиск уязвимостей в планах или коде.
*   **Reducing Hallucinations:** Снижение вероятности генерации ложных фактов за счет перекрестной проверки.

## Фаза 0: Основа и Настройка

### Шаг 0.1: Установка библиотек

**Что мы будем делать:**
Установим необходимые библиотеки для работы с графами и LLM.

In [ ]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-community

### Шаг 0.2: Настройка окружения

**Что мы будем делать:**
Загрузим API ключи и настроим модели. Обратите внимание, что мы используем модели с температурой 0 для Judge (чтобы быть последовательным) и температурой повыше (0.7) для спорщиков (для креативности).

In [ ]:
import os
import json
from typing import List, Dict, Any, Optional, Literal, TypedDict
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END

from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown

from notebook_utils import setup_environment

console = setup_environment(
    project_name="Agentic Architecture - Debate (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
)

# Модели
llm_creative = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0.7)
llm_strict = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0.0)

### Шаг 0.3: Модели данных для Дебатов

**Что мы будем делать:**
Определим структуры данных Pydantic, которые агенты будут использовать для обмена сообщениями:
1. **Argument:** Структура для аргумента Advocate (тезис, доказательство, источник, уверенность).
2. **Critique:** Структура для атаки Adversary (цель критики, слабое место, контраргумент).
3. **RoundJudgment:** Оценка судьи каждого раунда.
4. **FinalVerdict:** Итоговое решение.

In [ ]:
class Argument(BaseModel):
    """Структурированный аргумент со ссылкой на источник."""
    claim: str = Field(description="Основное утверждение")
    evidence: str = Field(description="Доказательства или факты")
    source: str = Field(description="Источник информации (URL или название)")
    confidence: float = Field(description="Уверенность в аргументе 0-1")

class Critique(BaseModel):
    """Критика аргумента."""
    target_claim: str = Field(description="Какое утверждение критикуется")
    weakness: str = Field(description="Обнаруженная слабость")
    source_validity: Literal["valid", "questionable", "invalid", "unverifiable"] = Field(
        description="Оценка источника"
    )
    counter_argument: str = Field(description="Контраргумент")
    counter_source: str = Field(description="Источник контраргумента")

class RoundJudgment(BaseModel):
    """Оценка раунда судьей."""
    advocate_score: int = Field(description="Оценка Advocate 1-10")
    adversary_score: int = Field(description="Оценка Adversary 1-10")
    round_winner: Literal["advocate", "adversary", "tie"] = Field(
        description="Победитель раунда"
    )
    key_points: List[str] = Field(description="Ключевые моменты раунда")
    feedback: str = Field(description="Обратная связь для обеих сторон")

class FinalVerdict(BaseModel):
    """Финальный вердикт по дебатам."""
    winner: Literal["advocate", "adversary", "tie"] = Field(
        description="Общий победитель"
    )
    advocate_total: int = Field(description="Общий балл Advocate")
    adversary_total: int = Field(description="Общий балл Adversary")
    conclusion: str = Field(description="Заключение по теме дебатов")
    fact_assessment: str = Field(description="Оценка фактической стороны дебатов")

class DebateState(TypedDict):
    topic: str
    advocate_position: str
    adversary_position: str
    rounds: List[Dict[str, Any]]  # {advocate_arg, adversary_critique, judgment}
    current_round: int
    max_rounds: int
    final_verdict: Optional[Dict]

print("Модели дебатов определены.")

## Фаза 1: Реализация Агентов

Теперь мы реализуем узлы графа для каждого из участников дебатов.

### Шаг 1.1: Агент Advocate (Защитник)

**Что мы будем делать:**
Реализуем функцию `advocate_node`, которая:
1. Анализирует текущую тему и критику из прошлого раунда (если есть).
2. Формирует новый аргумент, используя `llm_creative`.
3. Возвращает объект `Argument`.

In [ ]:
def advocate_node(state: DebateState):
    console.print(f"\n[bold green]--- ADVOCATE (Round {state['current_round'] + 1}) ---[/bold green]")
    
    # Контекст для агента
    prev_critique = "None (First round)"
    if state['rounds'] and 'adversary_critique' in state['rounds'][-1]:
        prev_critique = state['rounds'][-1]['adversary_critique']
    
    advocate_runner = llm_creative.with_structured_output(Argument)
    
    prompt = f"""
    You are the ADVOCATE supporting the position: "{state['advocate_position']}".
    Topic: {state['topic']}
    
    Previous critique you must address: {prev_critique}
    
    Present a strong argument. You MUST cite a source (even if simulated/general for this demo).
    Focus on facts and logic.
    """
    
    arg = advocate_runner.invoke(prompt)
    
    console.print(Panel(
        f"[bold]Claim:[/bold] {arg.claim}\n\n"
        f"[bold]Evidence:[/bold] {arg.evidence}\n\n"
        f"[bold]Source:[/bold] {arg.source}",
        title="Advocate's Argument",
        border_style="green"
    ))
    
    # Обновляем состояние: создаем новый раунд или обновляем текущий
    rounds = state['rounds'].copy()
    if len(rounds) <= state['current_round']:
        rounds.append({})
    
    rounds[state['current_round']]['advocate_arg'] = arg.model_dump()
    
    return {"rounds": rounds}

### Шаг 1.2: Агент Adversary (Критик)

**Что мы будем делать:**
Реализуем функцию `adversary_node`, которая:
1. Читает последний аргумент Advocate.
2. Ищет логические ошибки или сомнительные источники.
3. Формирует контраргумент, используя `Critique`.

In [ ]:
def adversary_node(state: DebateState):
    console.print(f"\n[bold red]--- ADVERSARY (Round {state['current_round'] + 1}) ---[/bold red]")
    
    current_arg = state['rounds'][state['current_round']]['advocate_arg']
    
    adversary_runner = llm_creative.with_structured_output(Critique)
    
    prompt = f"""
    You are the ADVERSARY opposing the position: "{state['advocate_position']}".
    You support: "{state['adversary_position']}"
    Topic: {state['topic']}
    
    Advocate's Argument to critique:
    Claim: {current_arg['claim']}
    Evidence: {current_arg['evidence']}
    Source: {current_arg['source']}
    
    Find weaknesses, check source credibility (simulate checking), and provide a counter-argument.
    """
    
    critique = adversary_runner.invoke(prompt)
    
    console.print(Panel(
        f"[bold]Target:[/bold] {critique.target_claim}\n\n"
        f"[bold]Weakness:[/bold] {critique.weakness}\n\n"
        f"[bold]Counter:[/bold] {critique.counter_argument}",
        title="Adversary's Critique",
        border_style="red"
    ))
    
    rounds = state['rounds'].copy()
    rounds[state['current_round']]['adversary_critique'] = critique.model_dump()
    
    return {"rounds": rounds}

### Шаг 1.3: Агент Judge (Судья)

**Что мы будем делать:**
Реализуем две функции для судьи:
1. **`judge_round_node`**: Оценивает текущий раунд, выставляя баллы.
2. **`judge_final_node`**: Выносит итоговый вердикт после окончания всех раундов.

In [ ]:
def judge_round_node(state: DebateState):
    console.print(f"\n[bold yellow]--- JUDGE (Round {state['current_round'] + 1}) ---[/bold yellow]")
    
    current_round_data = state['rounds'][state['current_round']]
    
    judge_runner = llm_strict.with_structured_output(RoundJudgment)
    
    prompt = f"""
    You are an impartial JUDGE.
    Topic: {state['topic']}
    
    Advocate Argument:
    {json.dumps(current_round_data['advocate_arg'])}
    
    Adversary Critique:
    {json.dumps(current_round_data['adversary_critique'])}
    
    Evaluate who won this round based on logic, evidence strength, and effective rebuttal.
    Assign scores 1-10.
    """
    
    judgment = judge_runner.invoke(prompt)
    
    console.print(Panel(
        f"[bold]Winner:[/bold] {judgment.round_winner.upper()}\n"
        f"Score: Advocate {judgment.advocate_score} - {judgment.adversary_score} Adversary\n\n"
        f"[bold]Feedback:[/bold] {judgment.feedback}",
        title="Round Judgment",
        border_style="yellow"
    ))
    
    rounds = state['rounds'].copy()
    rounds[state['current_round']]['judgment'] = judgment.model_dump()
    
    # Инкремент раунда
    return {
        "rounds": rounds,
        "current_round": state['current_round'] + 1
    }

def judge_final_node(state: DebateState):
    console.print(f"\n[bold yellow]--- FINAL VERDICT ---[/bold yellow]")
    
    final_judge_runner = llm_strict.with_structured_output(FinalVerdict)
    
    prompt = f"""
    Review the entire debate history and declare a winner.
    Topic: {state['topic']}
    
    History:
    {json.dumps(state['rounds'], indent=2)}
    
    Provide a comprehensive conclusion and fact assessment.
    """
    
    verdict = final_judge_runner.invoke(prompt)
    
    console.print(Panel(
        f"[bold]OVERALL WINNER:[/bold] {verdict.winner.upper()}\n"
        f"Total Score: Advocate {verdict.advocate_total} - {verdict.adversary_total} Adversary\n\n"
        f"[bold]Conclusion:[/bold] {verdict.conclusion}\n\n"
        f"[bold]Fact Assessment:[/bold] {verdict.fact_assessment}",
        title="🏆 Final Verdict",
        border_style="gold1"
    ))
    
    return {"final_verdict": verdict.model_dump()}

## Фаза 2: Сборка Графа

Соберем всех агентов в единый граф с циклической логикой раундов.

In [ ]:
def router(state: DebateState):
    if state['current_round'] >= state['max_rounds']:
        return "finalize"
    return "continue"

debate_graph = StateGraph(DebateState)

debate_graph.add_node("advocate", advocate_node)
debate_graph.add_node("adversary", adversary_node)
debate_graph.add_node("judge_round", judge_round_node)
debate_graph.add_node("judge_final", judge_final_node)

debate_graph.set_entry_point("advocate")

debate_graph.add_edge("advocate", "adversary")
debate_graph.add_edge("adversary", "judge_round")

debate_graph.add_conditional_edges(
    "judge_round",
    router,
    {
        "continue": "advocate",
        "finalize": "judge_final"
    }
)

debate_graph.add_edge("judge_final", END)

debate_agent = debate_graph.compile()
print("Debate Graph compiled.")

## Фаза 3: Демонстрация Дебатов

Запустим дебаты на сложную тему.

### Шаг 3.1: Запуск дебатов

**Что мы будем делать:**
Инициируем дебаты на тему "AI Safety vs AI Accelerationism" (Безопасность ИИ против Ускорения развития ИИ). Ограничим дебаты 2 раундами для краткости.

In [ ]:
topic = "Should AI development be paused to ensure safety mechanisms are in place?"

initial_state = {
    "topic": topic,
    "advocate_position": "Yes, we need a pause to implement safety guardrails and regulation.",
    "adversary_position": "No, pausing will only hinder progress and bad actors won't stop.",
    "rounds": [],
    "current_round": 0,
    "max_rounds": 2,
    "final_verdict": None
}

console.print(f"\nTopic: [bold white]{topic}[/bold white]\n")

debate_result = debate_agent.invoke(initial_state)

## Заключение

Архитектура **Debate + Red-teaming** продемонстрировала, как структурированный конфликт может привести к более глубокому анализу проблемы. Введение роли Судьи обеспечивает независимую оценку, а требование цитирования (даже симулированного) заставляет агентов быть более предметными.